In [1]:
%pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import psycopg2

In [4]:
# Conexão com o banco de dados padrão para criar o pokemon_db
connection = psycopg2.connect(
    dbname="postgres", user="postgres", password="1234", host="localhost"
)
connection.autocommit = True
cursor = connection.cursor()

# Criando o banco de dados
cursor.execute("CREATE DATABASE pokemon_db;")

In [5]:
# Fechando a conexão inicial
cursor.close()
connection.close()